<a href="https://colab.research.google.com/github/MariaInyutina/Parcing/blob/main/Parcing_extra_TG_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telethon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.2/620.2 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for telethon: filename=Telethon-1.37.0-py3-none-any.whl size=694365 sha256=a91a15e7d661b7c569a344c277a8f01f320b0ad2f8c60a654e4054fe28c2b9dd
  Stored in directory: /root/.cache/pip/wheels/8b/3c/0a/91d35499092d78120d71039a156b5c450f3ca00f95b4d594a4
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=4eea5ecfdd4d1ca6d854f1288ed7673d5955ca330aaed394e3697408d5e2279a
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built telethon pyaes


In [21]:
from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from datetime import datetime, timedelta, timezone
import pandas as pd

# Параметры для Telegram API
API_ID =
API_HASH = ''
CHANNEL_USERNAME = 'extrahse'  # Имя пользователя канала
phone = ''
session_name = 'parcing'
# Инициализация клиента
client = TelegramClient(session_name, API_ID, API_HASH)

# Функция для сбора статистики за определённый период
async def get_channel_stats(start_date, end_date):
    await client.start()
    stats = []

    # Получение истории сообщений канала за указанный период
    async for message in client.iter_messages(CHANNEL_USERNAME, offset_date=end_date):
        if message.date < start_date:
            break

        # Подсчет количества реакций
        reactions_count = sum(reaction.count for reaction in message.reactions.results) if message.reactions else 0

        # Сбор информации о сообщении
        data = {
            'date': message.date.strftime('%d-%m'),  # Форматируем дату как ДД-ММ
            'views': message.views,
            'reactions': reactions_count,
            'public_reposts': message.forwards if message.forwards else 0,
            'private_reposts': None,  # Telegram не предоставляет разделение репостов на публичные и непубличные
            'comments': message.replies.replies if message.replies else 0,
            'post_title': message.message[:30] if message.message else "Без текста",  # Первые 30 символов текста как заголовок
            'post_link': f'https://t.me/{CHANNEL_USERNAME}/{message.id}'  # Ссылка на пост
        }
        stats.append(data)

    # Сохранение данных в таблицу
    df = pd.DataFrame(stats)
    df.to_excel('telegram_extra_stats.xlsx', index=False)  # Сохранение в файл Excel
    print("Данные успешно сохранены в telegram_extra_stats.xlsx")

# Запрос ввода даты от пользователя
start_date_input = input("Введите начальную дату в формате ГГГГ-ММ-ДД: ")
end_date_input = input("Введите конечную дату в формате ГГГГ-ММ-ДД: ")

# Преобразование ввода в формат даты с временной зоной UTC
start_date = datetime.strptime(start_date_input, '%Y-%m-%d').replace(tzinfo=timezone.utc)
end_date = datetime.strptime(end_date_input, '%Y-%m-%d').replace(tzinfo=timezone.utc)

# Основная функция с использованием async with
async def main():
    async with client:
        await get_channel_stats(start_date, end_date)

# Запуск main() в асинхронной среде
await main()

Введите начальную дату в формате ГГГГ-ММ-ДД: 2024-10-28
Введите конечную дату в формате ГГГГ-ММ-ДД: 2024-11-01
Данные успешно сохранены в telegram_extra_stats.xlsx


**Использовать следующую ячейку при многоразовом запуске кода, нужна для завершения сеанса**

In [28]:
# Завершение клиента и пауза
await client.disconnect()
#await asyncio.sleep(1)  # Небольшая пауза для завершения фоновых задач

**Посмотреть получившуюсь таблицу**

In [22]:
df = pd.read_excel('telegram_extra_stats.xlsx')
df

,date,views,reactions,public_reposts,private_reposts,comments,post_title,post_link
0,31-10,2756,44,21,NaN,3,Не верю! 👻\n\nЭту фразу часто го,https://t.me/extrahse/2841
1,31-10,2354,19,6,NaN,2,Теперь в онлайне не только фри,https://t.me/extrahse/2840
2,30-10,2803,22,10,NaN,0,Вам оставили наследство наслед,https://t.me/extrahse/2838
3,30-10,2388,62,20,NaN,5,Без текста,https://t.me/extrahse/2837
4,29-10,2473,25,5,NaN,0,Your life — your rules!\n\nСовсе,https://t.me/extrahse/2836
5,28-10,2431,21,4,NaN,0,"Попробуй создать цифровой мир,",https://t.me/extrahse/2835
6,28-10,2593,33,9,NaN,2,Я вам запрещаю жарить яичницу:,https://t.me/extrahse/2834


**Преобразуем таблицу в привычный для нас вид**

In [24]:
# Переставляем столбцы в нужном порядке
df = df[['post_title', 'post_link', 'date', 'views', 'reactions', 'public_reposts', 'private_reposts', 'comments']]

# Транспонируем DataFrame, чтобы сделать столбцы строками
df_transposed = df.T

# Переименовываем индексы в соответствии с требуемыми названиями
df_transposed.index = [
    'Название поста',
    'Ссылка',
    'Дата',
    'Просмотры',
    'Реакции',
    'Репосты публичные',
    'Репосты непубличные',
    'Комментарии'
]

# Теперь df_transposed содержит нужную структуру
df_transposed

,0,1,2,3,4,5,6
Название поста,Не верю! 👻\n\nЭту фразу часто го,Теперь в онлайне не только фри,Вам оставили наследство наслед,Без текста,Your life — your rules!\n\nСовсе,"Попробуй создать цифровой мир,",Я вам запрещаю жарить яичницу:
Ссылка,https://t.me/extrahse/2841,https://t.me/extrahse/2840,https://t.me/extrahse/2838,https://t.me/extrahse/2837,https://t.me/extrahse/2836,https://t.me/extrahse/2835,https://t.me/extrahse/2834
Дата,31-10,31-10,30-10,30-10,29-10,28-10,28-10
Просмотры,2756,2354,2803,2388,2473,2431,2593
Реакции,44,19,22,62,25,21,33
Репосты публичные,21,6,10,20,5,4,9
Репосты непубличные,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Комментарии,3,2,0,5,0,0,2


**Скачиваем финальную таблицу**

In [27]:
df_transposed.to_excel('telegram_extra_stats.xlsx', index=True)
from google.colab import files

# Скачивание файла
files.download('telegram_extra_stats.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>